In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 10 - M. Massot 2022-2023 - Ecole polytechnique
#    ----------   
#    Loi de conservation scalaire non-linéaire : Burgers
#    
#    Auteurs : L. Séries et M. Massot - (C) 2022
#    

In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "seaborn"

# Loi de conservation scalaire : Burgers

## Equation de Burgers

La variable $u(t,x)$, $x\in\mathbb{R}$,  $t\in\mathbb{R}_+$, satisfait l'équation aux dérivées partielles :

$$\partial_t u + \partial_x(f(u)) = 0 $$

avec le flux non linéaire : 

$$f(u) = \frac{u^2}{2},$$

et le problème de Cauchy est complèté par la donnée initiale :

$$u(0,x) = u_0(x).$$

Dans ce notebook, nous considérons le problème du petit chapeau étudiée dans le polycopié et aussi un créneau qui permet de voir la solution faible entropique des deux problèmes de Riemann avec $U_g<U_d$, constituée d'une onde de détente, et avec $U_g>U_d$, constituée d'une onde de choc.

In [ ]:
# calcul du flux
def f(x):
    return (0.5*x*x)

In [ ]:
# Donnée initiale du petit chapeau
def hat_ini(x):
    u = np.empty(x.size)
    for ix, xi in enumerate(x):
        if xi <= -1 or xi>=1:
            u[ix] = 0
        elif xi < 0:
            u[ix] = xi + 1
        else:
            u[ix] = -xi + 1
    return u

# solution du petit chapeau pour l'équation de Burgers
def compute_uexa_hat(xini, xend, t):
    u = np.empty(5)
    u[0] = 0
    u[1] = 0
    u[2] = np.sqrt(2/(1+t))
    u[3] = 0
    u[4] = 0
    
    x = np.empty(5)
    x[0] = xini
    x[1] = -1
    x[2] = np.sqrt(2*(1+t)) - 1
    x[3] = np.sqrt(2*(1+t)) - 1
    x[4] = xend

    return x, u 

In [ ]:
# résolution du problème de Riemann en x/t = 0 pour deux états Ug et Ud à gauche et à droite - version scalaire
# Noté W_R(x/t=0,Ug,Ud) dans les notes écrites et les transparents de cours
#
def Riemann(Ug,Ud):
    Riem = Ug
    if (max(Ug,Ud)<=0) : Riem = Ud
    else :
        if Ug<=0 and Ud>=0 :
            Riem=0
        else :
            if Ug+Ud<=0 : Riem = Ud
    return Riem
#
# Il est possible de vectoriser de manière peu efficace
# avec la fonction np.vectorize
# 
Riemann_vec = np.vectorize(Riemann)

# Version vectorisée avec np.where plus performante
# utilisée dans le schéma de Godunov ci-dessous permettant une 
# résolution avec plusieurs milliers de cellules en quelques secondes
#
def Riemann2(Ug,Ud):
    Riem = Ug
    c = np.stack((Ug, Ud))
    Riem = np.where(np.amax(c,axis=0)<=0, Ud, Riem)
    Riem = np.where((Ug<=0) & (Ud>=0), 0, Riem)
    Riem = np.where((Ug>=0) & (Ud<=0) & ((Ug+Ud)<0), Ud, Riem)
    return Riem

### Schéma de Godunov

Le schéma de Godonov pour l'équation de Burgers est ici utilisé pour résoudre le problème du petit chapeau dans un premier temps. Nous avons vu en cours que le schéma, sous la conditions CFL permet de converger vers l'unique solution faible entropique du problème de Cauchy.

Un slider permet de voir l'influence de la discrétisation et la capacité du schéma à converger vers l'unique solution faible entropique étudiée dans le polycopié. On compare la solution résolue par le schéma à la solution exacte prédite par la théorie et on peut aussi fournir une courbe de convergence pour le schéma. 

In [ ]:
xmin = -2.0
xmax = 4.0

tini = 0.
tend = 6.
#
# Ne pas augmenter tend au dela de 11 car nous avons mis des conditions aux limites de type conditions périodiques
# mais la solution exacte n'est pas calculée en prenant en compte ce type de conditions aux limites.
# 

a = 1.

uini = hat_ini

cfl = 0.95

fig = go.Figure()
nxplot = 2000
dxplot = (xmax-xmin) / nxplot
xplot = np.linspace(xmin+(dxplot/2), xmax-(dxplot/2), nxplot)
fig.add_trace(go.Scatter(x=xplot, y=uini(xplot), name='initial sol.', line_color='rgb(140,140,140)'))

xexa10, uexa10 = compute_uexa_hat(xmin, xmax, tend/2)
xexa15, uexa15 = compute_uexa_hat(xmin, xmax, 3*tend/4)
xexa20, uexa20 = compute_uexa_hat(xmin, xmax, tend)

fig.add_trace(go.Scatter(x=xexa10, y=uexa10, mode='lines', name='exact sol. at t=0.5*tend', 
                         line=dict(dash='dash', color='rgb(76,114,176)')))
fig.add_trace(go.Scatter(x=xexa15, y=uexa15, mode='lines', name='exact sol. at t=0.75*tend', 
                         line=dict(dash='dash', color='rgb(221,132,82)')))
fig.add_trace(go.Scatter(x=xexa20, y=uexa20, mode='lines', name='exact sol. at t=tend', 
                         line=dict(dash='dash', color='rgb(85,168,104)')))

nx = [125, 250, 500, 1000, 2000]
for nx_i in nx:
        
    dx = (xmax-xmin) / nx_i
    x = np.linspace(xmin+dx/2, xmax-dx/2, nx_i)

    u = np.zeros(nx_i+1)
    u[1:] = uini(x)
    u[0] = u[-1]
    
    nt = int(a*(tend-tini)/(cfl*dx)) + 1
    dt = (tend-tini)/(nt-1)
    dtoverdx = dt/dx
    
    u10 = np.zeros(nx_i)
    u15 = np.zeros(nx_i)
    flux = np.zeros(nx_i)
    etats = np.zeros(nx_i)
    
    for it in range(nt-1):

        etats[:-1] = Riemann2(u[1:-1],u[2:])
        etats[-1] = Riemann(u[-1],u[0])
        flux = f(etats)

        u[2:] += -dtoverdx * (flux[1:] - flux[0:-1])
        u[1] += -dtoverdx * (flux[0] - flux[-1])
        u[0] = u[-1] #  conditions aux limites périodiques

        t = (it+1)*dt
        if it == nt//2: u10[:] = u[1:]
        if it == 3*(nt//4): u15[:] = u[1:]
            
    fig.add_trace(go.Scatter(x=x, y=u10, mode='lines', name='num. sol. at t=0.5*tend', 
                              line_color='rgb(76,114,176)', opacity=0.75, visible=False))
    fig.add_trace(go.Scatter(x=x, y=u15, mode='lines', name='num. sol. at t=0.75*tend', 
                             line_color='rgb(221,132,82)', opacity=0.75, visible=False))
    fig.add_trace(go.Scatter(x=x, y=u[1:], mode='lines', name='num. sol. at t=tend', 
                             line_color='rgb(85,168,104)', opacity=0.75, visible=False))

fig.data[4].visible = True  
fig.data[5].visible = True  
fig.data[6].visible = True

# Create and add slider
steps = []
for i, nx_i in enumerate(nx):
    args = [{"visible": [((el>=0 and el<=3)or (el>=3*i+4 and el<=3*i+6)) for el in range(len(fig.data))]}]
    ##print(args)
    step = dict(method="update", label = f"{nx_i}", args=args)
    steps.append(step)
sliders = [dict(currentvalue={'prefix': 'nx = '}, steps=steps)]
 
fig.update_layout(sliders=sliders, title=f"Schéma de Godunov : cfl={cfl}", height=500, 
                  legend=dict(x=0.75, y=0.95, bgcolor='rgba(0,0,0,0)'))

fig.show()